In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

**NOTA:** antes de correr las siguientes líneas de código, subir el fichero kaggle.json para establecer la conexión correctamente y descargar los csv correspondientes

## CARGA DE ARCHIVOS Y EXPLORACIÓN DE DATOS

In [76]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'

In [39]:
!kaggle competitions download -c udea-ai4eng-20241

udea-ai4eng-20241.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!unzip udea-ai4eng-20241.zip

Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [77]:
ds = pd.read_csv('train.csv')
dt = pd.read_csv('test.csv')
ds = ds.copy()

Muestra la tabla sin cambios:

In [78]:
display(ds)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,medio-alto
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) incompleta,bajo
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,medio-bajo
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,No,No,N,Si,Si,No,Primaria completa,bajo


## PREPROCESADO DE DATOS

Elimina las columnas que no nos sirven para el entrenamiento:

In [79]:
columns_drop = ['ID', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'FAMI_TIENEINTERNET.1', 'ESTU_PRIVADO_LIBERTAD']
ds = ds.drop(columns_drop, axis = 1)

Guarda las columnas actuales y las muestra:

In [80]:
col = ds.columns
print(col)

Index(['PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL'],
      dtype='object')


Muestra la cantidad de datos nulos (NaN) por cada columna antes de que sean eliminados:

In [81]:
for i in range (0, len(col)):
  print("missing data in column", col[i], ": ", sum(ds[col[i]].isna()))
  print("------------------------------------------------------------------")

missing data in column PERIODO :  0
------------------------------------------------------------------
missing data in column ESTU_PRGM_ACADEMICO :  0
------------------------------------------------------------------
missing data in column ESTU_PRGM_DEPARTAMENTO :  0
------------------------------------------------------------------
missing data in column ESTU_VALORMATRICULAUNIVERSIDAD :  6287
------------------------------------------------------------------
missing data in column ESTU_HORASSEMANATRABAJA :  30857
------------------------------------------------------------------
missing data in column FAMI_ESTRATOVIVIENDA :  32137
------------------------------------------------------------------
missing data in column FAMI_TIENEINTERNET :  26629
------------------------------------------------------------------
missing data in column FAMI_EDUCACIONPADRE :  23178
------------------------------------------------------------------
missing data in column ESTU_PAGOMATRICULAPROPIO :  6498

Elimina las filas que contienen al menos un valor nulo:

In [82]:
# Evalúa si cada columna tiene algún valor nulo y la elimina

def drop_null_rows(ds):
  df = ds.dropna()
  return df

df = drop_null_rows(ds)

Muestra la cantidad de datos nulos por cada columna después del proceso anterior:

In [83]:
for i in range (0, len(col)):
  print("missing data in column", col[i], ": ", sum(df[col[i]].isna()))
  print("------------------------------------------------------------------")

missing data in column PERIODO :  0
------------------------------------------------------------------
missing data in column ESTU_PRGM_ACADEMICO :  0
------------------------------------------------------------------
missing data in column ESTU_PRGM_DEPARTAMENTO :  0
------------------------------------------------------------------
missing data in column ESTU_VALORMATRICULAUNIVERSIDAD :  0
------------------------------------------------------------------
missing data in column ESTU_HORASSEMANATRABAJA :  0
------------------------------------------------------------------
missing data in column FAMI_ESTRATOVIVIENDA :  0
------------------------------------------------------------------
missing data in column FAMI_TIENEINTERNET :  0
------------------------------------------------------------------
missing data in column FAMI_EDUCACIONPADRE :  0
------------------------------------------------------------------
missing data in column ESTU_PAGOMATRICULAPROPIO :  0
---------------------

Muestra la tabla con las columnas y filas actualizadas:

In [84]:
display(df)

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,No,Si,Postgrado,medio-alto
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,No,Si,Técnica o tecnológica incompleta,bajo
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,No,No,Secundaria (Bachillerato) completa,bajo
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,No,Si,Secundaria (Bachillerato) completa,alto
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,No,Si,Primaria completa,medio-bajo
...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,Si,Secundaria (Bachillerato) incompleta,medio-alto
692496,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,No,Si,Secundaria (Bachillerato) incompleta,bajo
692497,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,Si,Secundaria (Bachillerato) incompleta,medio-bajo
692498,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,Si,Si,Primaria completa,bajo


Aplica conversión one-hot para cada columna:

In [85]:
def one_hot(df, column):
    # Obtener las columnas categóricas
    columnas_categoricas = column
    # Aplicar codificación one-hot a las columnas categóricas
    df_codificado = pd.get_dummies(df, columns=columnas_categoricas)

    return df_codificado
df_codificado,recuento = one_hot(df, col)
#cambia el corchete por el nombre de la columna que necesita

Pipeline después del preprocesado:

In [87]:
# Crear un pipeline que incluye el preprocesamiento y el modelo SVM
model = Pipeline(steps=[
    ('preprocessor', df_codificado),
    ('classifier', SVC(kernel='linear'))
])

## ESTABLECE LAS CARACTERÍSTICAS Y LA ETIQUETA PARA EL ENTRENAMIENTO Y PRUEBA:

In [88]:
# Separa las características (X) y etiquetas (y) en los datos de entrenamiento
X_train = df[['PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_EDUCACIONMADRE']]
y_train = df['RENDIMIENTO_GLOBAL']

# Separar características (X) en los datos de prueba
X_test = dt[['PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_EDUCACIONMADRE']]
# y_test = dt['RENDIMIENTO_GLOBAL']

## ENTRENAMIENTO SVM

In [98]:
print(X_train)
print(y_train)

[[20212 'ENFERMERIA' 'BOGOTÁ' ... 'No' 'Si' 'Postgrado']
 [20212 'DERECHO' 'ATLANTICO' ... 'No' 'Si'
  'Técnica o tecnológica incompleta']
 [20203 'MERCADEO Y PUBLICIDAD' 'BOGOTÁ' ... 'No' 'No'
  'Secundaria (Bachillerato) completa']
 ...
 [20183 'ADMINISTRACIÓN EN SALUD OCUPACIONAL' 'BOGOTÁ' ... 'Si' 'Si'
  'Secundaria (Bachillerato) incompleta']
 [20195 'PSICOLOGIA' 'TOLIMA' ... 'Si' 'Si' 'Primaria completa']
 [20195 'PSICOLOGIA' 'ANTIOQUIA' ... 'No' 'Si'
  'Técnica o tecnológica completa']]
['medio-alto' 'bajo' 'bajo' ... 'medio-bajo' 'bajo' 'alto']


In [95]:
# Convertir los datos de entrada a arrays de Numpy
X_train = X_train.values()
y_train = y_train.values()
# Asegurarse de que las etiquetas sean unidimensionales
y_train = y_train.ravel()

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [93]:
# Entrenar el modelo
model.fit(X_train, y_train)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().